## Hello, Here's How to use RAG w HF Models

Install some dependencies

In [41]:
!pip install -q -U bitsandbytes==0.42.0
!pip install -q -U peft==0.8.2
!pip install -q -U trl==0.7.10
!pip install -q -U accelerate==0.27.1
!pip install -q -U datasets==2.17.0
!pip install -q -U transformers==4.38.1
!pip install langchain sentence-transformers chromadb langchainhub

!pip install langchain-community langchain-core

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.38.1 which is incompatible.
  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached transformers-4.49.0-py3-none-any.whl (10.0 MB)
Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1


Get the Model You Want

In [42]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# get the repository ID for the Gemma 2b model which am testing with
repo_id = "google/gemma-3-1b-it"

Define Variables

In [43]:
import os

# set your own hf token then fetch it here
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

# obv params, max_length is max token len for generated text, temp=0.1 means give more predictable and less random results
llm = HuggingFaceEndpoint(
    task='text-generation',
    repo_id=repo_id,
    model="google/gemma-3-1b-it",
    max_length=1024,
    temperature=0.1,
    huggingfacehub_api_token=hf_token
)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


Define Data Sources

In [70]:
import pandas as pd

# load ur data
health_data = pd.read_csv('./sample_data/data-with-sources.csv')
work_data = pd.read_csv('./sample_data/work-and-education-data.csv')

health_data_sample = health_data
work_data_sample = work_data

health_data_sample['text'] = health_data_sample['Question'].fillna('') + ' ' + health_data_sample['Answer'].fillna('')
work_data_sample['text'] = work_data_sample['Theme'].fillna('') + ' ' + work_data_sample['Content'].fillna('')

In [47]:
# forgot one dependency
!pip install chromadb

Set Embedding Model, and Chroma Client to Interact w Vector Database and Create Collections

In [71]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import chromadb

# pt model for geenrating embeddings used pretty often
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# persistent client to interact w chroma vector store
client = chromadb.PersistentClient(path="./chroma_db")

# create collections for each data (for testing rn)
health_collection = client.get_or_create_collection(name="health_docs")
work_collection = client.get_or_create_collection(name="work_docs")

Function to add data to collection by embedding them

In [72]:
def add_data_to_collection(collection, data):
    for idx, row in data.iterrows():
        try:
            # get the embeddings using the embedding model for the documents
            embeddings = embedding_model.embed_documents([row['text']])[0]
            collection.add(
                ids=[str(idx)],
                embeddings=[embeddings],
                documents=[row['text']]
            )
        except Exception as e:
            print(f"Error on index {idx}: {e}")

# add data to collections
add_data_to_collection(health_collection, health_data_sample)
add_data_to_collection(work_collection, work_data_sample)

Function to now match for releveant document

In [74]:
def get_relevant_document(query, category):
    try:
        # get the embedding for the user query using same embedding model
        query_embeddings = embedding_model.embed_documents([query])[0]

        # choose the correct collection based on the category
        collection = health_collection if category == "health" else work_collection

        # query the collection
        results = collection.query(query_embeddings=[query_embeddings], n_results=1)

        print(f"Query Results: {results}")

        return results['documents'][0][0] if results['documents'] else None
    except Exception as e:
        print(f"Error querying: {e}")
        return None

Generate Answer

In [75]:
def generate_answer(query, category):
    # b4 rag
    output_before_rag = llm.predict(f"Respond to this question: {query}")
    response_before_rag = output_before_rag

    # get the relevant document
    relevant_document = get_relevant_document(query, category)
    if relevant_document is None:
        return f"Sorry, no relevant document found. Model's response before RAG: {response_before_rag}"

    relevant_document = " ".join(relevant_document.split())
    MAX_DOC_LENGTH = 500
    relevant_document = relevant_document[:MAX_DOC_LENGTH]

    rag_prompt = f"""
    You are a helpful assistant for international students new to B.C. Here is a relevant document:

    {relevant_document}

    Please respond to the following question based on the document above:

    Question: {query}

    Answer:
    """

    print("Prompt being sent to model:")
    print(rag_prompt)

    # now generate using RAG
    output_after_rag = llm.predict(rag_prompt)
    print("Output from model:", output_after_rag)

    response_after_rag = output_after_rag

    # return both responses to compare
    return {
        "Before RAG Response": response_before_rag,
        "After RAG Response": response_after_rag
    }

Example Usage

In [76]:
user_query = "What do I need to do to apply for MSP coverage in B.C.?"
category = "health"
responses = generate_answer(user_query, category)

print("User Query:", user_query)
print("Response Before RAG:", responses["Before RAG Response"])
print("Response After RAG:", responses["After RAG Response"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_

Query Results: {'ids': [['8']], 'embeddings': None, 'documents': [['How do I apply for MSP? You can apply online, in person, or by submitting a paper form through the B.C. Application for Health and Drug Coverage.']], 'uris': None, 'data': None, 'metadatas': [[None]], 'distances': [[0.40488243103027344]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
Prompt being sent to model:

    You are a helpful assistant for international students new to B.C. Here is a relevant document:

    How do I apply for MSP? You can apply online, in person, or by submitting a paper form through the B.C. Application for Health and Drug Coverage.

    Please respond to the following question based on the document above:

    Question: What do I need to do to apply for MSP coverage in B.C.?

    Answer: 
    
Output from model:  - You can apply online, in person, or by submitting a paper form through the B.C. Application for He

In [77]:
# verify
health_docs = health_collection.get()
print("Number of documents in health collection:", len(health_docs['documents']))

work_docs = work_collection.get()
print("Number of documents in work collection:", len(work_docs['documents']))

Number of documents in health collection: 76
Number of documents in work collection: 878
